# Hyperparameter search across optimizer, learning rate, weight decay

In [3]:
# Global imports
import os
import json
import sys
sys.path.insert(0, 'src')

In [8]:
# Local imports
from utils import read_json, write_json
from train import main as train_fn
from parse_config import ConfigParser

In [6]:
# architectures = [
#     "vgg11_bn",
#     "vgg13_bn",
#     "vgg16_bn",
#     "vgg19_bn",
#     "resnet18",
#     "resnet34",
#     "resnet50",
#     "densenet121",
#     "densenet161",
#     "densenet169",
#     "mobilenet_v2",
#     "googlenet",
#     "inception_v3"]
print(os.getcwd())

config_path = 'configs/cinic10_imagenet_train_hyperparameter_search.json'
config_dict = read_json(config_path)

# optimizers = ["Adam", "SGD"]
# learning_rates = [0.1, 0.01, 1e-3, 1e-4, 1e-5, 1e-6]
# weight_decays = [0, 0.1, 0.01, 1e-3, 1e-4]
# n_epochs = 30

# Test values
architectures = ["vgg11_bn"]
learning_rates = [0.1, 0.01]
weight_decays = [0, 0.1]
n_epochs = 30



/n/fs/ac-editing/model-editing


In [17]:
for architecture in architectures:
    config_dict.update({"name": "HyperParameterSearch_{}".format(architecture)})
    # Update model architecture information
    for key, value in config_dict["arch"]["args"].items():
        config_dict["arch"]["args"].update({key: value.format(architecture)})

    print(config_dict["name"])
    print(config_dict["arch"])
    for optimizer in optimizers:
        config_dict["optimizer"].update({"type": optimizer})
        for learning_rate in learning_rates:
            config_dict["optimizer"]["args"].update({"lr": learning_rate})
            for weight_decay in weight_decays:
                config_dict["optimizer"]["args"].update({"weight_decay": weight_decay})
                print(config_dict["optimizer"])
                run_id = "{}-{}-optim_{}-lr_{}-wd_{}".format(
                    config_dict["data_loader"]["type"],
                    config_dict["arch"]["args"]["type"],
                    config_dict["optimizer"]["type"],
                    config_dict["optimizer"]["args"]["lr"],
                    config_dict["optimizer"]["args"]["weight_decay"]
                )
                # Create config object
                config = ConfigParser(config_dict, run_id=run_id)
                train_fn(config)

'''
for each arch:
    change config name=HyperParamSearch_{arch.type}, arch.type, arch.checkpoint_path,
    for each optimizer:
        set config optimizer.type
        for each lr:
            set config optimizer.args.lr
            for each weight decay:
                setconfig.optimizer.weight_decay
                config.run_id = {dataset}_{arch.type}_{optim}_{lr}_{weight_decay}
                config = ConfigParser(config)
                train.main(config)
                

'''


HyperParameterSearch_vgg11_bn
OrderedDict([('type', 'CIFAR10PretrainedModel'), ('args', OrderedDict([('type', 'vgg11_bn'), ('checkpoint_path', 'external_code/PyTorch_CIFAR10/cifar10_models/state_dicts/vgg11_bn.pt')]))])
OrderedDict([('type', 'Adam'), ('args', OrderedDict([('lr', 0.1), ('weight_decay', 0), ('amsgrad', False)]))])
Created train and validation dataloaders for CINIC10DataLoader
Train data folder: data/cinic-10-imagenet/train
Validation data folder: data/cinic-10-imagenet/valid
Created CIFAR10PretrainedModel model with 28149514 trainable parameters
Restored weights from external_code/PyTorch_CIFAR10/cifar10_models/state_dicts/vgg11_bn.pt
Checkpoint save directory: saved/HyperParameterSearch_vgg11_bn/CINIC10DataLoader-vgg11_bn-optim_Adam-lr_0.1-wd_0/models
Train Epoch: 1 [0/70000 (0%)] Loss: 1.778389
Train Epoch: 1 [4096/70000 (6%)] Loss: 2.351776


In [8]:
# print stuff
print(config["optimizer"])

OrderedDict([('type', 'Adam'), ('args', OrderedDict([('lr', 1e-06), ('weight_decay', 0), ('amsgrad', False)]))])
